In [16]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [17]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [18]:
params = {"grupo":"7"}

In [19]:
r = requests.get(base_url, params=params)

In [20]:
page = BeautifulSoup(r.text, 'html.parser')

In [21]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [22]:
__EVENTVALIDATION

'/wEdAAwQ6vZgiYV2bVqVZUu/B/Zx4mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc3k+UPboGEbut3ELnSncv0Xkm63sA=='

In [23]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [24]:
__VIEWSTATE

'...'

In [25]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [26]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [27]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [28]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [29]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [30]:
print(md5(a_tags[0].text))

66b0019727446b81e6667622e30b2874


In [31]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [32]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [33]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 62/62 [00:00<00:00, 38749.34it/s]


In [34]:
databases

[{'database_id': '66b0019727446b81e6667622e30b2874',
  'database_title': '156 - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/156/2018-10-01_156_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 1)},
 {'database_id': 'fcd48779a785811b55d386a5f9863c10',
  'database_title': '156 - Historico - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/156/2018-10-01_156_-_Historico_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 1)},
 {'database_id': 'c3d7c7962b0f35bdb3a4f1561341db99',
  'database_title': 'legisladoc - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/Legisladoc/2018-10-12_legisladoc_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 12)},
 {'database_id': 'd50a4a71caff5d8508db98202652768a',
  'database_title': 'Sigmu - CARACTERISTICA GEO - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/sigmu/2018-10-12_Sigmu_-_CAR

In [38]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/sigmu/2018-10-12_Sigmu_-_PERIODO_PENDENCIA_-_Base_de_Dados.csv', encoding="ISO-8859-1", delimiter=';')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
test.fillna('').head()

,SIS_DATA_INI,SSO_IDF,PPE_DATA_FIM,UFU_IDF
0,------------,-------,------------,-------
1,2016-01-01 13:39:11.100,629462,2016-01-05 16:20:00.130,3
2,2016-01-01 20:29:11.763,629463,2016-01-11 13:15:23.257,8
3,2016-01-01 21:49:11.270,629464,2016-01-05 16:05:23.093,86
4,2016-01-01 22:19:11.180,629465,2016-01-04 16:05:54.620,5
